In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()
pp.pprint(f'{env.nS = }')
pp.pprint(f'{env.nA = }')
pp.pprint(f'{env.action_space = }')
pp.pprint(f'{env.observation_space = }')
pp.pprint(f'{env.shape = }')
pp.pprint(f'{env.reward_range = }')
pp.pprint(f'{env.spec = }')
pp.pprint(f'{env.P[0] = }')


'env.nS = np.int64(16)'
'env.nA = 4'
'env.action_space = Discrete(4)'
'env.observation_space = Discrete(16)'
'env.shape = [4, 4]'
'env.reward_range = (-inf, inf)'
'env.spec = None'
('env.P[0] = {0: [(1.0, 0, 0.0, True)], 1: [(1.0, 0, 0.0, True)], 2: [(1.0, 0, '
 '0.0, True)], 3: [(1.0, 0, 0.0, True)]}')


In [3]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.

    V(s)=s′∑​P(s′∣s,π(s))[R(s,π(s),s′)+γV(s′)]
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    iteration = 0
    while True:
        iteration += 1
        diff = 0
        # for each action, do full backup
        for s in range(env.nS):
            current_v = 0
            # for all possible next actions
            for a, action_prob in enumerate(policy[s]):
                # for each action, look at the reward and possible next state
                for prob, ns, r, done in env.P[s][a]:
                    # calculate the expected value
                    current_v += action_prob * prob * (r + discount_factor * V[ns])
            # check the max val change across all states
            diff = max(diff, np.abs(current_v - V[s]))        
            V[s] = current_v
        # make sure the max val change is smaller than theta before we stop
        if(diff < theta):
            break
    # pp.pprint(f'{V = }')
    # pp.pprint(f'{iteration = }')
    return np.array(V)

In [8]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """

    # given state s, get how good it is for s if taking each action
    def compute_action_values(V, s):
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, ns, r, done in env.P[s][a]:
                A[a] += prob * (r + discount_factor * V[ns])
        return A

    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    # pp.pprint(f'{policy = }')

    while True:
        # evaluate the current policy
        V = policy_eval_fn(policy, env, discount_factor)
        # assume policy is stable, update to false when there is a difference in the current policy with the previous policy
        policy_stable = True

        for s in range(env.nS):
            # get current action with current policy
            current_a = np.argmax(policy[s])
            best_a = np.argmax(compute_action_values(V, s))

            if current_a != best_a:
                policy_stable = False

            policy[s] = np.eye(env.nA)[best_a]
            # pp.pprint(f'{policy = }')

        if policy_stable:    
            return policy, V

In [14]:
import time
start_time = time.perf_counter()
policy, v = policy_improvement(env)
end_time = time.perf_counter()
print(f"Time taken: {(end_time - start_time)*1000} mseconds")

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Time taken: 10.045170999546826 mseconds
Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [11]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)